In [11]:
import rasterio
import os
import glob
import pandas as pd
from PIL import Image
from numpy import asarray
import numpy as np
from scipy import stats
import matplotlib as plt
import cv2

In [12]:
# Get files
dir_spot = "/home/simon/CDE_UBS/thesis/data_collection/spot6/"
dir_sen = "/home/simon/CDE_UBS/thesis/data_collection/sen2/merged_reprojected/"

def find_files(path,critera):
    search_term = os.path.join(path, critera)
    files = glob.glob(search_term)
    return(files)

In [15]:
def get_stats(filepaths):
    counter = 0
    file_stats_dict = []
    file_stats_ls = []
    
    for file in filepaths:
        stats_file_dict = {}
        stats_file_ls = {}
        
        #im = cv2.imread(file)
        #im = im.astype(int)
        with rasterio.open(file, 'r') as ds:
            arr = ds.read()  # read all raster values
            arr = np.transpose(arr, (1,2,0)) 
        
        # catch errors for faulty images(?)
        try:
            b1 = arr[:,:,0].flatten()
            b2 = arr[:,:,1].flatten()
            b3 = arr[:,:,2].flatten()

            # remove invalid data
            b1 = b1[(b1 > 0) & (b1 <= 9999)]
            b2 = b2[(b2 > 0) & (b2 <= 9999)]
            b3 = b3[(b3 > 0) & (b3 <= 9999)]

            # append to return dict
            stats_file_dict["b1"] = [len(b1),int(np.sum(b1)),round(np.nanmean(b1),3),round(np.nanstd(b1),3)]
            stats_file_dict["b2"] = [len(b2),int(np.sum(b2)),round(np.nanmean(b2),3),round(np.nanstd(b2),3)]
            stats_file_dict["b3"] = [len(b3),int(np.sum(b3)),round(np.nanmean(b3),3),round(np.nanstd(b3),3)]
            file_stats_dict.append(stats_file_ls)
            # append everyting to return ls (for potential pd df)
            file_stats_ls.append([file,"B1",len(b1),int(np.sum(b1)),round(np.nanmean(b1),3),round(np.nanstd(b1),3)])
            file_stats_ls.append([file,"B2",len(b2),int(np.sum(b2)),round(np.nanmean(b2),3),round(np.nanstd(b2),3)])
            file_stats_ls.append([file,"B3",len(b3),int(np.sum(b3)),round(np.nanmean(b3),3),round(np.nanstd(b3),3)])
        except IndexError:
            pass
       
    # print progress
        counter = counter+1
        print(counter,"/",len(filepaths)," finished                ",end="\r")
        
        """
        # implement early stop for testing
        if counter==5:
            print("\nearly stop")
            return(pd.DataFrame(pd.DataFrame(file_stats_ls,columns=["path","band","len","sum","mean","stdev"])))
        """
    return(pd.DataFrame(pd.DataFrame(file_stats_ls,columns=["path","band","len","sum","mean","stdev"])))

In [16]:
# find files
files_spot = find_files(dir_spot,"*.jp2")
files_sen  = find_files(dir_sen,"*.tif")


# claculate stats and save
stats_spot = get_stats(files_spot[344:])
stats_spot.to_pickle("stats_spot.pkl")
stats_sen = get_stats(files_sen)
stats_spot.to_pickle("stats_sen.pkl")

KeyboardInterrupt: 